In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("drive/MyDrive/Colab Notebooks/CS7650/final")
os.listdir()

## Libraries

In [ ]:
import pandas as pd
import json
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import tqdm
from ast import literal_eval

## GPU check

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

## Vocab

In [ ]:
class Vocab:
  def __init__(self, w2i_file, wc_file, min_count=1):
    with open(w2i_file, 'r') as file:
      self.word2id = json.load(file)
    with open(wc_file, 'r') as file:
      self.word_counts = json.load(file)
    self.num_words = len(self.word2id.keys())
    self.min_count = min_count
    self.infrequent = [k for k,v in self.word_counts.items() if v <= min_count]
  
  def _word2id(self, word, train):
    if train and (word in self.infrequent and random.random() > 0.5):
      return 0
    else:
      return self.word2id.get(word, 0)

  def sentence2indices(self, sentence, train):
    return [self._word2id(word, train) for word in sentence.split()]
    #return [[self._word2id(word, train) for word in s.split()] for s in sentences]

In [ ]:
w2i_file = 'data/vocab/word2id.json'
wc_file = 'data/vocab/word_counts.json'
vocab = Vocab(w2i_file, wc_file)

In [ ]:
with open('data/gloVe/filtered_glove.json', 'r') as file:
  gloVe = json.load(file)
  file.close()

## Data

In [ ]:
num_chunks = 4 #start with 1/4 of data for now
df_trains = []
df_tests = []
for i in range(num_chunks):
  temp_train = pd.read_csv('data/clean_indexed/train_clean_id{0}.csv'.format(i))
  temp_train['comment_text'] = temp_train['comment_text'].apply(literal_eval)
  temp_test = pd.read_csv('data/clean_indexed/test_clean_id{0}.csv'.format(i))
  temp_test['comment_text'] = temp_test['comment_text'].apply(literal_eval)
  df_trains.append(temp_train)
  df_tests.append(temp_test)
df_train = pd.concat(df_trains, axis=0, ignore_index=True)
df_test = pd.concat(df_tests, axis=0, ignore_index=True)
df_train.head()

In [ ]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
X_all = df_train['comment_text'].tolist()
Y_all = df_train[classes].values.tolist()
X_train, X_val, Y_train, Y_val = train_test_split(X_all, Y_all, test_size=0.25, random_state=1)
# X_train = df_train['comment_text'].tolist()
# Y_train = df_train[classes].values.tolist()
X_test = df_test['comment_text'].tolist()
Y_test = df_test[classes].values.tolist()

In [ ]:
print(f'{len(X_train)} rows in train')
print(f'{len(X_val)} rows in validation')
print(f'{len(X_test)} rows in test')

In [ ]:
for i in range(len(classes)):
  print(f'class {classes[i]}')
  u_train, c_train = np.unique(np.array(Y_train)[:,i], return_counts=True)
  print(u_train, c_train / c_train.sum())
  u_val, c_val = np.unique(np.array(Y_val)[:,i], return_counts=True)
  print(u_val, c_val / c_val.sum())
  print('----')

## CNN BiLSTM with Attention
- https://ieeexplore.ieee.org/document/8577620
  - ok this one still makes zero sense to me so 
- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8784247
  - is bi lstm into cnn lol (Reverse from above)
  - also more recent (2019 vs 2019)
- bruh ok that one doesnt make sense either when it comes to attention
  - https://dl.acm.org/doi/pdf/10.1145/3097983.3098140

### old

In [ ]:
class CNN_BiLSTM(nn.Module):
  def __init__(self, VOCAB_SIZE, DIM_EMB=300, NUM_CLASSES=6, gloVe=None, dropout=0, num_filters=30):
    super(CNN_BiLSTM, self).__init__()
    self.NUM_CLASSES=NUM_CLASSES
    self.num_words = VOCAB_SIZE + 1#forgot padding character lol
    self.pad_idx = VOCAB_SIZE# + 1
    
    #Embedding
    self.embed = nn.Embedding(num_embeddings=self.num_words, embedding_dim=DIM_EMB, padding_idx=self.pad_idx)
    if gloVe:
      #weights = torch.zeros_like(self.embed.weight)
      weights = torch.normal(0,1,size=self.embed.weight.shape)
      for w, e in gloVe.items():
        w_idx = vocab.word2id[w]
        weights[w_idx] = torch.FloatTensor(e)
      self.embed.weight.data.copy_(weights)   
    #CNN
    self.num_filters = num_filters
    kernel_sizes = [1,2,3] #unigrams, bigrams, trigrams
    self.conv = nn.ModuleList([nn.Conv2d(in_channels=1
                                         ,out_channels=self.num_filters
                                         ,kernel_size=(k, DIM_EMB)) for k in kernel_sizes])
    self.relu = nn.ReLU() 
    #BiLSTM
    self.lstm = nn.LSTM(input_size=len(kernel_sizes) * self.num_filters
                        ,hidden_size=1
                        ,batch_first=True
                        ,dropout=dropout
                        ,bidirectional=True)  
    #Attention
    #self.W_a = nn.Linear()
    #Final
    self.final = nn.Sigmoid()

  def forward(self, X, mask):
    #(batch size, 1, max len, DIM_EMB)
    embed = self.embed(X).unsqueeze(1)
    #each conv is (batch size, num_filters, height, width)
    #ok but essentially this is conv1d right so it's functionally (batch_size, channels=num_filters, L_out)
    #it returns at first like (batch_size, channels=num_filters, L_out, 1)
    convs = [self.relu(c(embed)).squeeze(-1) for c in self.conv]
    #max pooling (max value in width at each height?)
    #each conv is then (batch size, height)?
    #mp = [torch.max(c, dim=-1)[0].squeeze(1) for c in convs]
    mp = [F.max_pool1d(c, c.size(2)).squeeze(2) for c in convs] # [(batch size, num_filters)]?
    #concatenate to create (batch size, 3 (uni-bi-tri), height)
    h = torch.cat(mp, dim=1) # (batch size, n_kernels * len(kernel_sizes))
    #input to lstm needs to be (seq len, batch size, input size)
    return

### New

In [ ]:
class BiLSTM_CNN(nn.Module):
  def __init__(self, VOCAB_SIZE, DIM_EMB=300, DIM_HID=300, DIM_FC=100, NUM_LAYERS=1, NUM_CLASSES=6, gloVe=None, dropout=0.2, num_filters=30):
    super(BiLSTM_CNN, self).__init__()
    self.NUM_CLASSES=NUM_CLASSES
    self.num_words = VOCAB_SIZE + 1#forgot padding character lol
    self.pad_idx = VOCAB_SIZE# + 1
    self.DIM_HID = DIM_HID
    self.DIM_FC = DIM_FC
    self.num_layers = NUM_LAYERS
    self.dropout = nn.Dropout(dropout)
    
    #Embedding
    self.embed = nn.Embedding(num_embeddings=self.num_words, embedding_dim=DIM_EMB, padding_idx=self.pad_idx)
    if gloVe:
      #weights = torch.zeros_like(self.embed.weight)
      weights = torch.normal(0,1,size=self.embed.weight.shape)
      for w, e in gloVe.items():
        w_idx = vocab.word2id[w]
        weights[w_idx] = torch.FloatTensor(e)
      self.embed.weight.data.copy_(weights)   
    #BiLSTM
    self.lstm = nn.LSTM(input_size=DIM_EMB
                        ,hidden_size=self.DIM_HID
                        ,batch_first=True
                        ,num_layers=self.num_layers
                        ,bidirectional=True) 
    #CNN
    self.num_filters = num_filters
    kernel_sizes = [1,2,3] #unigrams, bigrams, trigrams
    self.conv = nn.ModuleList([nn.Conv2d(in_channels=1
                                         ,out_channels=self.num_filters
                                         ,kernel_size=(k, self.DIM_HID * 2)) for k in kernel_sizes])
                                         #,kernel_size=(k, self.DIM_HID)) for k in kernel_sizes])
    self.relu = nn.ReLU() 
    
    #Attention
    # self.w_a = nn.Linear(in_features=self.DIM_HID
    #                      ,out_features=self.DIM_HID)
    
    #Final
    # self.linear = nn.Linear(in_features=len(kernel_sizes) * self.num_filters
    #                         ,out_features=self.NUM_CLASSES)
    # self.linear = nn.Linear(in_feature=self.DIM_HID
    #                         ,out_features=self.NUM_CLASSES)
    self.fc1 = nn.Linear(in_features=len(kernel_sizes)*self.num_filters
                         ,out_features=self.DIM_FC)
    # self.fc1 = nn.Linear(in_features=len(kernel_sizes)*self.num_filters + 2 * self.DIM_HID
    #                      ,out_features=self.DIM_FC)
    self.fc2 = nn.Linear(in_features=self.DIM_FC
                         ,out_features=self.NUM_CLASSES)
    self.final = nn.Sigmoid()

  def forward(self, X, mask=None, train=True):
    #X, mask -> (batch size, max len)
    #embed
    embed = self.embed(X.cuda()) #(batch size, max len, DIM_EMB)
    embed = self.dropout(embed)

    #packed
    source_lengths = torch.sum(X != self.pad_idx, axis=1).cpu()
    embed = nn.utils.rnn.pack_padded_sequence(embed, lengths=source_lengths, batch_first=True, enforce_sorted=False)

    #lstm
    output, (h_n, c_n) = self.lstm(embed) #(batch size, seq len, num_directions * hidden_size)
    hidden, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True, padding_value=0.0)
    hidden = self.dropout(hidden)
    #hidden = output[:,:,self.DIM_HID:] + output[:,:,:self.DIM_HID] #just add them lol? kinda unclear how the paper handles it
    
    #CNN
    #hidden should be (batch size, channels=1, seq len, hidden size)
    convs = [self.relu(c(hidden.unsqueeze(1))).squeeze(-1) for c in self.conv] #each conv is (batch size, num filters, seq len)
    #print(convs[0].shape)
    mp = [F.max_pool1d(c, c.size(2)).squeeze(2) for c in convs] # [(batch size, num_filters)]?
    m = torch.cat(mp, dim=1) # (batch size, num_filters * len(kernel_sizes))
    
    #attention
    #last_hidden = torch.zeros_like(hidden)
    #last_hidden[:, 1:, :] = hidden[:, :-1, :]
    
    #(batch size, hidden dim, seq len) x (batch size, seq len, hidden dim)
    #u_t = torch.bmm(last_hidden.transpose(1,2), self.w_a(hidden)) #(batch size, hidden dim, hidden dim)... this dont seem right
    
    #(batch size, seq len, hidden dim) x (batch size, hidden dim, seq_len)
    #u = torch.bmm(last_hidden, self.w_a(hidden).transpose(1,2)) #(batch size, seq len, seq len) this literally makes no sense how does it multiply with m
    # u = self.w_a(hidden) #(batch size, seq len)

    #a needs to be (batch len, seq len)
    # a = torch.matmul(u.transpose(0,1), m) #(seq len, num_filters * len(kernel_sizes))
    # a = F.softmax(a)
    # s = hidden * a

    #F.softmax()
    out = m
    # out = torch.cat((m, hidden[:,-1,:].squeeze(1)), dim=1)

    fc1 = F.relu(self.fc1(out))
    return self.fc2(fc1)
    #return self.final(linear)

## Train

In [ ]:
def EvalNet(net, X, Y, threshold=0.5, verbose=True):
  net.eval()
  pred = np.zeros_like(Y)
  for i in tqdm.notebook.tqdm(range(len(X)), leave=False):
    x = torch.LongTensor(X[i]).unsqueeze(0)
    #idk what to do when the seq length is less than 3 lol
    if (x.size(-1) < 3):
      pred[i] = np.array([0,0,0,0,0,0])
      continue
    #probs = net.forward(x, train=False).cpu()
    probs = torch.sigmoid(net.forward(x, None, train=False)).cpu()
    pred[i] = np.array(probs > threshold, dtype=float)
  # x, x_mask = pad_input(X, net.pad_idx)
  # probs = net.forward(x, x_mask)
  # pred = np.array(probs > threshold, dtype=float)
  Y = np.array(Y)
  for i in range(len(classes)):
    #print(Y[:,i])
    #print(pred[:,i])
    acc = accuracy_score(Y[:,i], pred[:,i])
    rec = recall_score(Y[:,i], pred[:,i])
    prec = precision_score(Y[:,i], pred[:,i])
    f1 = f1_score(Y[:,i], pred[:,i])
    if verbose:
      print(f'{classes[i]} label')
      print(f'Accuracy: {acc} Recall {rec} Precision {prec} F1 {f1}')
      print('-----------------------')
  total_acc = accuracy_score(Y, pred)
  total_rec = recall_score(Y, pred, average='micro')
  total_prec = precision_score(Y, pred, average='micro')
  total_f1 = f1_score(Y, pred, average='micro')
  if verbose:
    print('Total')
    print(f'Accuracy: {total_acc} Recall {total_rec} Precision {total_prec} F1 {total_f1}')
  return total_f1

def shuffle_sentences(sentences, tags):
  shuffled_sentences = []
  shuffled_tags = []
  indices = list(range(len(sentences)))
  random.shuffle(indices)
  for i in indices:
    shuffled_sentences.append(sentences[i])
    shuffled_tags.append(tags[i])
  return (shuffled_sentences, shuffled_tags)

#Pad inputs to max sequence length (for batching)
def pad_input(X_list, pad_val):
  X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list], batch_first=True, padding_value=pad_val).type(torch.LongTensor)
  X_mask = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list], batch_first=True).type(torch.FloatTensor)
  return X_padded, X_mask

def Train(net, X, Y, max_iter, lr, PATH):
  print("Start Training!")
  optimizer = optim.Adam(net.parameters(), lr=lr)

  num_classes = len(classes)
  batch_size = 50

  best_f1 = 0.0
  f1_dec_ct = 0

  for epoch in range(max_iter):
    total_loss = 0.0
    net.train()   #Put the network into training model
    (X_shuffled, Y_shuffled) = shuffle_sentences(X, Y)
    for batch in tqdm.notebook.tqdm(range(0, len(X), batch_size), leave=False):
      x, x_mask = pad_input(X_shuffled[batch:batch+batch_size], net.pad_idx)
      y = torch.FloatTensor(Y_shuffled[batch:batch+batch_size]).cuda()

      net.zero_grad()
      probs = net.forward(x, x_mask, train=True)

      #crit = nn.BCELoss()
      crit = nn.BCEWithLogitsLoss()
      loss = crit(probs, y)
      total_loss += loss

      loss.backward()
      optimizer.step()
    print(f'loss on epoch {epoch} = {total_loss}')
    #validation
    f1_val = EvalNet(net, X_val, Y_val, verbose=False)
    print(f'validation f1-score: {f1_val}')
    if f1_val > best_f1:
      best_f1 = f1_val
      torch.save(net.state_dict(), PATH)
      f1_dec_ct = 0
    #early stopping
    else:
      f1_dec_ct += 1
      if f1_dec_ct == 5:
        break

## Model

In [ ]:
PATH = 'Models/BiLSTM_CNN.pt'
num_filters = 50
net = BiLSTM_CNN(VOCAB_SIZE=vocab.num_words, gloVe=gloVe, num_filters=num_filters).cuda()
Train(net, X_train, Y_train, max_iter=50, lr=0.001, PATH=PATH)

In [ ]:
net = BiLSTM_CNN(VOCAB_SIZE=vocab.num_words, gloVe=gloVe, num_filters=num_filters).cuda()
net.load_state_dict(torch.load(PATH))
EvalNet(net, X_test, Y_test)